In [1]:
import json
import pandas as pd
import numpy as np
import math
from scipy import optimize
from datetime import datetime
import prettytable as pt
from matplotlib import pyplot as plt
import os
import prettytable as pt
import warnings
warnings.filterwarnings("ignore")



###################################################################################################
                                                                                                  #
from rhoova.Client import *                                                                       #   
#Register and get api key from https://app.rhoova.com/ for ClientConfig("api key", "api secret")  #
config = ClientConfig("", "")                                                                     #
api = Api(config)                                                                                 # 
                                                                                                  #
###################################################################################################


directory = os.path.normpath(os.getcwd() + os.sep + os.pardir + os.sep + os.pardir)
yielddatadirectory=directory+"/data/yielddata/bonddefiniton.csv"
marketdatadirectory=directory+"/data/marketdata/marketdata.csv"


bonddefiniton = pd.read_csv(yielddatadirectory)
marketdata = pd.read_csv(marketdatadirectory)

data=bonddefiniton.merge(marketdata[['valuationDate','isinCode','value']], left_on='isinCode', right_on='isinCode')


In [2]:
bonddata ={
  "notional": 1000000,
  "settlementDate": "2021-03-09",
  "valuationDate": "2021-03-05",
  "buySell": "Buy",
  "zeroCouponBondDefinition": {
    "businessDayConvention": "Unadjusted",
    "calendar": "Turkey",
    "currency": "USD",
    "dateGeneration": "Backward",
    "dayCounter": "Thirty360E",
    "endOfMonth": True,
    "issueDate": "2014-02-19",
    "maturityDate": "2045-02-17",
    "maturityDateConvention": "Unadjusted",
    "redemption": 100
  },
  "discountCurve": {
    "calendar": "NullCalendar",
    "currency": "USD",
    "dayCounter": "Thirty360E",
    "instruments": {
      "BONDS": {
        "method": "NelsonSiegel",
        "numberOfIterations": 10000,
        "accuracy": 1e-06,
        "simplexLambda": 10,
        "settlementDays": 2,   
      }
    },
    "intpMethod": "Linear",
    "period": "3M",
    "settlementDays": 2
  },
  "yieldData": data.to_dict('r')
}


In [3]:
try:
    bondresult = api.createTask(CalculationType.ZERO_COUPON_BOND, bonddata,True)
    result=json.loads(bondresult["result"])
except RhoovaError as e:
    e.printPretty()    

In [4]:
cashflows=pd.DataFrame(result.get("data"))
cashflows=cashflows[cashflows["termToMatByDay"]>0]
cashflows

,fixingDate,accrualStart,accrualEnd,notional,currency,leg,payOrReceive,instrument,rate,zeroRate,spread,termToMatByDay,termToMatByYear,cashflow,discountFactor,cashflowPv
0,2014-02-17,2014-02-19,2045-02-17,1000000,USD,Fixed,Buy,Zero Coupon Bond,0,0.068148,0,8618,23.938889,1000000,0.195511,195511.277824


In [5]:
npvTable = pt.PrettyTable(['Parameters', 'Value'])
npvTable.add_row(['PV', result.get('pv')])
npvTable.add_row(['Clean Price', result.get('cleanPrice')])
npvTable.add_row(['Dirty Price', result.get('dirtyPrice')])
npvTable.add_row(['Accrued Amount', result.get('accruedAmount')])
npvTable.align = 'r'
npvTable.float_format = '.3'
print(npvTable)




+----------------+------------+
|     Parameters |      Value |
+----------------+------------+
|             PV | 195511.278 |
|    Clean Price |     19.557 |
|    Dirty Price |     19.557 |
| Accrued Amount |          0 |
+----------------+------------+
